In [1]:
class numerics:
    class shift():
        def __init__(self, plus, mnus):
            self.plus = plus
            self.mnus = mnus

        def __radd__(self, arg):
            return type(arg)(
                arg.start + self.plus,
                arg.stop + self.plus
            )

        def __rsub__(self, arg):
            return type(arg)(
                arg.start - self.mnus,
                arg.stop - self.mnus
            )
        
        def __rmod__(self, arg):
            return type(arg)(
                arg.start - self.mnus,
                arg.stop + self.plus
            )
        
        def __rmul__(self, arg):
            return type(self)(
                arg * self.plus,
                arg * self.mnus
            )

    def __init__(self):
        self.one = self.shift(1,1)
        self.hlf = self.shift(0,1)
        self.eps = 1e-8
    
    def F(self, psi_l, psi_r, GC):
        return np.maximum(0, GC) * psi_l + np.minimum(0, GC) * psi_r
    
    def flux(self, psi, GC, ih):
        o = self.one
        h = self.hlf
        i = ih + h # TODO !!! (dziala, rozumiemy, ale brzydkie)

        return self.F(psi[i], psi[i+o], GC[i+h])
    
    def upwind(self, psi, flx, G, n, i):        
        h = self.hlf
        psi[n+1][i] = psi[n][i] - (flx[i+h] - flx[i-h]) / G[i]

    def ndx_psi(self, opts, psi, i):
        o = self.one
        e = self.eps

        result = 2 *(psi[i+o] - psi[i]) 
        
        if not opts["iga"]:
            result /= (psi[i+o] + psi[i] + e)
        else:
            result /= (1+1)
            
        return result
        
    def dfl(self, opts, GC, G, psi, i):
        h = self.hlf
        o = self.one

        result = -.5 * GC[i+h] / (G[i+o] + G[i]) * (GC[i+o+h] - GC[i-h])
        
        if not opts["iga"]:
            result *= .5 *  (psi[i+o] + psi[i]) 

        return result
        
    def GC_antidiff(self, opts, psi, GC, G, ih):
        h = self.hlf
        o = self.one
        i = ih + h # TODO !!! (dziala, rozumiemy, ale brzydkie)
        
        result = (
            GC[i+h] - GC[i+h]**2 
            / 
            (.5 *(G[i+o] + G[i]))  
        ) * .5 * self.ndx_psi(opts, psi, i)
                
        if opts["dfl"]:
            result += self.dfl(opts, GC, G, psi, i)
            
        return result
    
    def fct_extremum(self, a1, a2, a3, a4, extremum):
        return extremum(extremum(extremum(a1, a2), a3), a4)
    
    def fct_running_extremum(self, psi, i, extremum):
        o = self.one
        a1 = psi[i-o] 
        a2 = psi[i]
        a3 = psi[i+o]
        
        return extremum(extremum(a1, a2), a3)
    
    def fct_running_maximum(self, psi, i):
        return self.fct_running_extremum(psi, i, np.maximum)

    def fct_running_minimum(self, psi, i):
        return self.fct_running_extremum(psi, i, np.minimum)

    def fct_beta_up(self, psi, psi_max, flx, G, i):
        e = self.eps
        o = self.one
        h = self.hlf
        
        return (
            (self.fct_extremum(psi_max[i], psi[i-o], psi[i], psi[i+o], np.maximum) - psi[i]) * G[i]
        ) / (
            np.maximum(flx[i-h], 0)
            - np.minimum(flx[i+h], 0)
            + e
        )
    
    def fct_beta_dn(self, psi, psi_min, flx, G, i):
        e = self.eps
        o = self.one
        h = self.hlf
        
        return (
            (psi[i] - self.fct_extremum(psi_min[i], psi[i-o], psi[i], psi[i+o], np.minimum)) * G[i]
        ) / (
            np.maximum(flx[i+h], 0)
            - np.minimum(flx[i-h], 0)
            + e
        )